In [1]:
import numpy  as np
import pandas as pd
import data_tools
import matplotlib.pyplot as plt

from scipy.stats import norm
from Models.models import *

## Some functions 

In [2]:
ts = pd.read_csv('Preprocessed_data/ts.csv')

log_diff_prices = ts.price.values
log_diff_prices = np.array(log_diff_prices, dtype=float)
log_diff_prices = np.diff(np.log(log_diff_prices))
log_diff_prices = np.concatenate((np.array([0]), log_diff_prices))
ts['log_diff_prices'] = log_diff_prices

yearmonth = list(ts['yearmonth'])
x_label   = data_tools.from_index_to_dates(yearmonth)

ts['X'] = np.zeros(ts.shape[0])

for t in ts.index:
    if t == 0:
        continue
        
    curr_price = ts.loc[t  , 'price']
    prev_price = ts.loc[t-1, 'price'] 
    
    ts.loc[t,'X'] = residual(prev_price,curr_price,t)

eta = 0.5
zd  = 105.69451612903225
r   = 0.05

discount = 1/(1+r)

elev_stor = zip(ts.elevation.values,ts.storage.values)
elev_stor = sorted(elev_stor, key=lambda x: (x[1],x[0]))

def get_middle_points(x):
    x = np.array(x)
    return (x[:-1] + x[1:])/2


def pi_fun(storage, inflow, t, P, q, model_inflow, model_evaporation):
    
    expected_storage = model_storage(model_inflow, inflow , model_evaporation, t, storage, q) # this may not be needed if forecast is done.
    
    return eta * q * ((z(storage, elev_stor) + z(expected_storage,elev_stor))/2 - zd) * P

#def inter_0_extremes()

## Define inputs

### Grids

In [95]:
N = 10 # For the storage grid
M = 10 # For the price grid
L = 10 # For the outflow grid
S = 3 # For the states


P_min = 0.020
P_max = 0.160

I_min = 2000
I_max = 3000

q_min = 0
q_max = 245

price_grid   = get_middle_points(np.arange(P_min, P_max+ 2**-15, (P_max - P_min)/M))
storage_grid = get_middle_points(np.arange(I_min, I_max+ 2**-15, (I_max - I_min)/N))

rho = 2.33
K = 10 # number of shocks in the storage grid
J = 10 # number of shocks in the prices grid

shocks_storage  = get_middle_points(np.arange(-rho, rho+2**-15, (2*rho)/ K))

dist_shocks_storage = (shocks_storage[1] - shocks_storage[0])/2
prob_shocks_storage = norm.cdf(shocks_storage[:] + dist_shocks_storage) - norm.cdf(shocks_storage[:] - dist_shocks_storage)

shocks_prices  = get_middle_points(np.arange(-rho, rho+2**-15, (2*rho)/ J))

dist_shocks_prices = (shocks_prices[1] - shocks_prices[0])/2
prob_shocks_prices = norm.cdf(shocks_prices[:] + dist_shocks_prices) - norm.cdf(shocks_prices[:] - dist_shocks_prices)



outflow_grid = np.arange(q_min, q_max+ 2**-15, (q_max - q_min)/L) # L+1 elements

V0 = np.zeros((N,M))
V1 = np.zeros((N,M))

policy = np.zeros((S,N,M))

### Forecast inflow and storage

In [4]:
time = ts.shape[0]
ts_forecast = ts.copy()

for i in range(N):
    
    current_time = time + i
    
    prev_inflow  = ts_forecast.loc[current_time - 1, 'inflow' ]
    prev_storage = ts_forecast.loc[current_time - 1, 'storage']
    
    current_storage = model_storage(model_inflow,prev_inflow, model_evaporation, current_time, prev_storage, prev_inflow)
    current_inflow  = model_inflow(current_time, prev_inflow)
    
    ts_forecast.loc[current_time,'storage'] = current_storage
    ts_forecast.loc[current_time,'inflow']  = current_inflow

### Forecast prices

In [39]:
# Forecasting Prices M steps ahead
for i in range(M):
    
    current_time = time + i
    
    prev     = ts_forecast.loc[current_time-1,  'price']
    prev_x   = ts_forecast.loc[current_time-1,  'X']
    prev_y_x = ts_forecast.loc[current_time-12, 'X']
    
    log_P      = model_log_prices(current_time,prev, prev_x, prev_y_x)
    curr_price = np.exp(log_P)
    
    ts_forecast.loc[current_time,'price'] = curr_price
    
    X = residual(prev,curr_price,current_time)
    ts_forecast.loc[current_time,'X'] = X

## Algorithm 

In [6]:
pi_call = lambda storage, inflow, t, P, q: pi_fun(storage, inflow, t, P, q, model_inflow, model_evaporation)

In [96]:
max_ite = 3
ite     = 0
tol     = 2**-3

while ite < max_ite:
    
    for s in range(S):
        current_time = time + s
        for n in range(N):
            inflow       = ts_forecast.loc[current_time,'inflow']
            storage      = storage_grid[n]
#             storage      = ts_forecast.loc[current_time,'storage']
#             print(storage)

            for m in range(M):
                price = price_grid[m]
                actions = []
                for q in outflow_grid:
                    input_next_storage = [model_inflow, ts_forecast.loc[current_time - 1,'inflow'] , model_evaporation, t,
                                          storage, q, model_storage]
                    input_next_price   = [current_time, price]

                    reward = pi_call(storage, inflow, current_time, price, q) + discount * future_payments(V0,shocks_storage, prob_shocks_storage, shocks_prices, 
                                                                                                           prob_shocks_prices, input_next_storage, input_next_price, 
                                                                                                           price_grid, storage_grid)
                    actions.append(reward)

                policy[s,n,m]  = outflow_grid[np.argmax(actions)]
                V1[n,m]      = np.max(actions) 

    if np.max(np.abs(V0-V1)) < tol:
        V0 = V1.copy()
        break
        
    V0   = V1.copy()
    ite += 1
    print(ite)

1
2
3


In [97]:
policy

array([[[ 49. ,  49. ,  49. ,  49. ,  49. ,  49. ,  49. ,  49. ,  49. ,
          49. ],
        [ 98. ,  98. ,  98. ,  98. ,  98. ,  98. ,  98. ,  98. ,  98. ,
          98. ],
        [ 98. ,  98. ,  98. ,  98. ,  98. ,  98. ,  98. ,  98. ,  98. ,
          98. ],
        [147. , 147. , 147. , 147. , 147. , 147. , 147. , 147. , 147. ,
         147. ],
        [196. , 196. , 196. , 196. , 196. , 196. , 196. , 196. , 196. ,
         196. ],
        [220.5, 220.5, 220.5, 220.5, 220.5, 220.5, 220.5, 220.5, 220.5,
         220.5],
        [220.5, 220.5, 220.5, 147. , 220.5, 220.5, 220.5, 220.5, 220.5,
         220.5],
        [220.5, 220.5, 220.5, 220.5, 220.5, 220.5, 220.5, 220.5, 220.5,
         220.5],
        [220.5, 220.5, 220.5, 220.5, 220.5, 220.5, 220.5, 220.5, 220.5,
         220.5],
        [220.5, 220.5, 220.5, 220.5, 220.5, 220.5, 220.5, 220.5, 220.5,
         220.5]],

       [[ 24.5,  24.5,  24.5,  24.5,  24.5,  24.5,  24.5,  24.5,  24.5,
          24.5],
        [ 98. ,  98

In [84]:
# This function has a wide margin for acceleration

def future_payments(V,shocks_storage, probs_shock_storage, shocks_price, probs_shock_price, input_next_storage, input_next_price, price_grid, storage_grid):
    K = len(shocks_storage)
    J = len(shocks_price  )
    
    future_payment = 0
    
    for k in range(K):
        for j in range(J):
            future_storage = next_storage(*input_next_storage, shocks_storage[k]) 
            future_price   = next_price(  *input_next_price  , shocks_price[j])
            
            x,y = nearest(future_storage, storage_grid, future_price, price_grid) # this can be refined
            
            payment = V[x,y] * probs_shock_price[j] * probs_shock_storage[k]
            
            future_payment += payment
            
            if future_storage < 2000:
                future_payment = - 1000
    
    return future_payment

In [8]:
def nearest(next_storage, storage_grid, next_price, price_grid):
    x = np.argmin(np.abs(storage_grid - next_storage))
    y = np.argmin(np.abs(price_grid - next_price))
    
    return x, y

In [32]:
np.argmin(np.abs(price_grid-0.07))

3

In [33]:
price_grid[3]

0.069